In [2]:
# Import des librairies
import requests
import re

import openmeteo_requests

import pandas as pd
import numpy as np
import requests_cache
from retry_requests import retry
import time


In [3]:
df = pd.read_csv('df_voies_vertes.csv')
df.head()



,Unnamed: 0,Nom,Lien,Début,Fin,Distance,Type voie,Nature voie,Revêtement,Carte,Coordonnées de début,Coordonnées de fin,Coordonnées milieu
0,0,Voie Verte du Chemin d’Encochebérot,https://www.af3v.org/les-voies-vertes/voies/12...,L ISLE JOURDAIN,L ISLE JOURDAIN,"1,6km",Voie verte lisse,voie verte,Enrobé,https://sig.af3v.org/index.php/view/embed/?rep...,"('1.09673702748082', '43.62109781579031')","('1.1131950619939002', '43.6267673096849')","(1.1049660447373602, 43.62393256273761)"
1,1,Voie Verte du canal de Saint-Martory,https://www.af3v.org/les-voies-vertes/voies/12...,CUGNAUX,TOURNEFEUILLE,"2,1km",Voie verte lisse,voie verte,Enrobé,https://sig.af3v.org/index.php/view/embed/?rep...,"('1.32295597087284', '43.5323431084653')","('1.3433622170553097', '43.55502022774329')","(1.333159093964075, 43.5436816681043)"
2,2,Voie Verte de la Masse Pont-du-Casse-Agen,https://www.af3v.org/les-voies-vertes/voies/11...,PONT DU CASSE,AGEN,"2,3km",Voie verte lisse,Voie Verte,Enrobé,https://sig.af3v.org/index.php/view/embed/?rep...,"('0.6480315531906747', '44.216304129821')","('0.6696608866837562', '44.22602284001947')","(0.6588462199372155, 44.22116348492023)"
3,3,Voie Verte des trois ponts Castelnau d’Estréte...,https://www.af3v.org/les-voies-vertes/voies/12...,CASTELNAU D ESTRETEFONDS,FRONTON,"3,2km",Voie verte lisse,Voie Verte,Enrobé,https://sig.af3v.org/index.php/view/embed/?rep...,"('1.35618301280646', '43.76616910015008')","('1.3659247959472003', '43.78084206988599')","(1.3610539043768302, 43.773505585018036)"
4,4,Voie Verte des Berges de l'Ill à Mulhouse,https://www.af3v.org/les-voies-vertes/voies/13...,MULHOUSE,SAUSHEIM,"3,3km",Voie verte lisse,Canal,Enrobé lisse,https://sig.af3v.org/index.php/view/embed/?rep...,"('7.34639', '47.773977')","('7.362202', '47.79706')","(7.354296, 47.7855185)"


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 321 entries, 0 to 320
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Unnamed: 0            321 non-null    int64 
 1   Nom                   321 non-null    object
 2   Lien                  321 non-null    object
 3   Début                 321 non-null    object
 4   Fin                   321 non-null    object
 5   Distance              321 non-null    object
 6   Type voie             321 non-null    object
 7   Nature voie           315 non-null    object
 8   Revêtement            317 non-null    object
 9   Carte                 321 non-null    object
 10  Coordonnées de début  321 non-null    object
 11  Coordonnées de fin    321 non-null    object
 12  Coordonnées milieu    321 non-null    object
dtypes: int64(1), object(12)
memory usage: 32.7+ KB


In [7]:
def solar_info(coord_string):
    """
    # Conversion de l'adresse en points géo (lat/lon)

    url1 = f"https://api-adresse.data.gouv.fr/search/?q={adress.replace(' ', '+')}"
    response1 = requests.get(url1)
    data = response1.json()
    if data['features']:
        lon, lat = data['features'][0]['geometry']['coordinates']
    """
    coord_string = coord_string.strip('()')  # Retire les parenthèses si présentes
    lat, lon = map(float, coord_string.split(','))

    # Délai de 1-2 secondes entre chaque requête
    time.sleep(1.5)
    
    #lon, lat = colonne
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    # Paramètres à renseigner
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": f"2024-01-01",
        "end_date": f"2024-12-31",
        "daily": ["sunshine_duration", "temperature_2m_mean"],
        "hourly": "shortwave_radiation",
        "timezone": "Europe/Berlin"
    }
    responses = openmeteo.weather_api(url, params=params)

    response = responses[0]
    

    # Process hourly data
    hourly = response.Hourly()
    hourly_shortwave_radiation = hourly.Variables(0).ValuesAsNumpy()

    hourly_data = {"date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    )}

    hourly_data["shortwave_radiation"] = hourly_shortwave_radiation

    hourly_dataframe = pd.DataFrame(data = hourly_data)


    # Nettoyage pour exploitation des données du df Hourly
    hourly_dataframe['month'] = hourly_dataframe['date'].dt.month_name(locale='English')
    hourly_dataframe['month_num'] = hourly_dataframe['date'].dt.month

    hourly_dataframe = hourly_dataframe.iloc[2:]

    df_ghi = hourly_dataframe.groupby(['month', 'month_num']).agg(
        ghi_mean_W_m2=('shortwave_radiation', 'mean')
    )

    df_ghi = df_ghi.sort_values(by='month_num')


    # Process daily data
    daily = response.Daily()
    daily_sunshine_duration = daily.Variables(0).ValuesAsNumpy()
    daily_temperature_2m_mean = daily.Variables(1).ValuesAsNumpy()

    daily_data = {"date": pd.date_range(
        start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
        end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = daily.Interval()),
        inclusive = "left"
    )}

    daily_data["sunshine_duration"] = daily_sunshine_duration
    daily_data["temperature_2m_mean"] = daily_temperature_2m_mean

    daily_dataframe = pd.DataFrame(data = daily_data)


    # Nettoyage pour exploitation des données du df daily
    daily_dataframe['sunshine_duration_hours'] = daily_dataframe['sunshine_duration'] / 3600
    daily_dataframe.drop(columns='sunshine_duration', inplace=True)

    daily_dataframe = daily_dataframe.iloc[1:]

    daily_dataframe['month'] = daily_dataframe['date'].dt.month_name(locale='English')
    daily_dataframe['month_num'] = daily_dataframe['date'].dt.month

    df_weather = daily_dataframe.groupby(['month', 'month_num']).agg(
        temp_mean_c=('temperature_2m_mean', 'mean'),
        sunshine_duration_mean_hour=('sunshine_duration_hours', 'mean')
    )

    df_weather = df_weather.sort_values(by='month_num')

    # Concaténation des dfs

    df = pd.merge(df_weather, df_ghi, on='month')
    
    #===============#
    #    Formules   #
    #===============#

    # Calcul du ghi_journalier
    df['ghi_journalier'] = (df['sunshine_duration_mean_hour'] * df['ghi_mean_W_m2']) / 1000

    # Inclinaison du panneau, standard à 15°, servant de toit sur les portions de pistes cyclables lors de mauvais temps
    df['inclinaison'] = np.sin(np.radians(15))

    # Correction de la température, car rendement baisse au dela de 25°c
    df['correction_temp'] = 1 - (df['temp_mean_c'] - 25) * 0.004

    # Formule pour production energie / jours 
    df['energie_jour_kWh/j/m2'] = round(df['ghi_journalier'] * df['inclinaison'] * df['correction_temp'],2)

    # Multiplication pour production energie / mois 
    df['energie_mois_kWh/mois/m2'] = round(df['energie_jour_kWh/j/m2'] * 30.41,2)

    # Création d'un df de simulation
    df_copy = df.copy()

    df_copy.drop(columns=['temp_mean_c', 'sunshine_duration_mean_hour', 'ghi_mean_W_m2', 'ghi_journalier', 'inclinaison', 'correction_temp'], inplace=True)

    df_copy['prod_1m2_mensuel'] = df_copy['energie_mois_kWh/mois/m2']
    df_copy['prod_5m2_mensuel'] = df_copy['energie_mois_kWh/mois/m2'] * 5
    df_copy['prod_10m2_mensuel'] = df_copy['energie_mois_kWh/mois/m2'] * 10
    df_copy['prod_20m2_mensuel'] = df_copy['energie_mois_kWh/mois/m2'] * 20


    # Concaténation des dfs

    df_final = pd.merge(df, df_copy, on='month')

    df_final.drop(columns=['energie_jour_kWh/j/m2_x', 'energie_mois_kWh/mois/m2_x'], inplace=True)
    df_final.rename(columns={
        'energie_jour_kWh/j/m2_y':'energie_jour_kWh/j/m2',
        'energie_mois_kWh/mois/m2_y' : 'energie_mois_kWh/mois/m2'
    }, inplace=True)


    # Calcul production annuelle 
    # Calcul de la production annuelle pour chaque surface
    """
    df_annuel = pd.DataFrame({
        "Énergie produite annuelle (kWh)":
            round(df_final['energie_mois_kWh/mois/m2'].sum(), 2),
        
    })
    """


    """
    print(f"Données concernant ces coordonnées : ")
    print(f"Latitude : {lat}")
    print(f"Latitude : {lon}")
    print(f"Données concernant cette année : ")
    print(f"2024")
    """
    #return df_annuel
    #return df_final
    return round(df_final['energie_mois_kWh/mois/m2'].sum(), 2)
    

In [8]:
df['Energie produite annuelle (kWh)'] = df['Coordonnées milieu'].apply(solar_info)

In [12]:
df.rename(columns={
    'Energie produite annuelle (kWh)' : 'Énergie produite annuelle (kWh / m2 de panneau)'
}, inplace=True)

In [13]:
df.head()

,Unnamed: 0,Nom,Lien,Début,Fin,Distance,Type voie,Nature voie,Revêtement,Carte,Coordonnées de début,Coordonnées de fin,Coordonnées milieu,Énergie produite annuelle (kWh / m2 de panneau)
0,0,Voie Verte du Chemin d’Encochebérot,https://www.af3v.org/les-voies-vertes/voies/12...,L ISLE JOURDAIN,L ISLE JOURDAIN,"1,6km",Voie verte lisse,voie verte,Enrobé,https://sig.af3v.org/index.php/view/embed/?rep...,"('1.09673702748082', '43.62109781579031')","('1.1131950619939002', '43.6267673096849')","(1.1049660447373602, 43.62393256273761)",246.32
1,1,Voie Verte du canal de Saint-Martory,https://www.af3v.org/les-voies-vertes/voies/12...,CUGNAUX,TOURNEFEUILLE,"2,1km",Voie verte lisse,voie verte,Enrobé,https://sig.af3v.org/index.php/view/embed/?rep...,"('1.32295597087284', '43.5323431084653')","('1.3433622170553097', '43.55502022774329')","(1.333159093964075, 43.5436816681043)",242.65
2,2,Voie Verte de la Masse Pont-du-Casse-Agen,https://www.af3v.org/les-voies-vertes/voies/11...,PONT DU CASSE,AGEN,"2,3km",Voie verte lisse,Voie Verte,Enrobé,https://sig.af3v.org/index.php/view/embed/?rep...,"('0.6480315531906747', '44.216304129821')","('0.6696608866837562', '44.22602284001947')","(0.6588462199372155, 44.22116348492023)",273.37
3,3,Voie Verte des trois ponts Castelnau d’Estréte...,https://www.af3v.org/les-voies-vertes/voies/12...,CASTELNAU D ESTRETEFONDS,FRONTON,"3,2km",Voie verte lisse,Voie Verte,Enrobé,https://sig.af3v.org/index.php/view/embed/?rep...,"('1.35618301280646', '43.76616910015008')","('1.3659247959472003', '43.78084206988599')","(1.3610539043768302, 43.773505585018036)",244.18
4,4,Voie Verte des Berges de l'Ill à Mulhouse,https://www.af3v.org/les-voies-vertes/voies/13...,MULHOUSE,SAUSHEIM,"3,3km",Voie verte lisse,Canal,Enrobé lisse,https://sig.af3v.org/index.php/view/embed/?rep...,"('7.34639', '47.773977')","('7.362202', '47.79706')","(7.354296, 47.7855185)",249.67


In [14]:
df.to_csv("BDD_PC_PotentielSolaire_Annuel.csv", index=False)